In [0]:
# Imports
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.types import StringType,BooleanType,DateType
from pyspark.sql.functions import *

import json

# Error Messaging

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

## Kakfa Consumer Setup

In [0]:


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "new-songs"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe(['new-songs'])

In [0]:
aString = ""

kafkaListDictionaries = []

for i in range(100):
    try:
        msg = c.poll(timeout=15)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            aString['Timestamp'] = msg.timestamp()[1]
            kafkaListDictionaries.append(aString)
            c.commit()
    except Exception as e:
        print(e)


print(json.dumps(kafkaListDictionaries, indent=4, sort_keys=True))

[
    {
        "Acousticness": 0.16300000250339508,
        "ArtistID": "3hYtANQYrE6pd2PbtEyTIy",
        "Danceability": 0.8510000109672546,
        "DurationMS": 172600,
        "Energy": 0.7950000166893005,
        "Instrumentalness": 0.0,
        "Liveness": 0.2070000022649765,
        "Loudness": -9.02400016784668,
        "Mode": 1,
        "MusicalKey": 9,
        "Popularity": 53,
        "ReleaseYear": 1993,
        "Speechiness": 0.14499999582767487,
        "Tempo": 108.04900360107422,
        "TimeSignature": 4,
        "Timestamp": 1664317100503,
        "TrackID": "1Vtiq4XS36Y0qvFp9HRyin",
        "TrackName": "El Maestro",
        "Valence": 0.8389999866485596,
        "artists": "['Los Tigres Del Norte']"
    },
    {
        "Acousticness": 8.130000060191378e-05,
        "ArtistID": "2yEwvVSSSUkcLeSTNyHKh8",
        "Danceability": 0.36000001430511475,
        "DurationMS": 322017,
        "Energy": 0.796999990940094,
        "Instrumentalness": 0.8050000071525574,
  

In [0]:
#Save the consumed messages into a dictionary

k_list_dict = kafkaListDictionaries.copy()
songs_consumed = spark.createDataFrame(k_list_dict)

In [0]:
#Display the spark dataframe
display(songs_consumed)

Acousticness,ArtistID,Danceability,DurationMS,Energy,Instrumentalness,Liveness,Loudness,Mode,MusicalKey,Popularity,ReleaseYear,Speechiness,Tempo,TimeSignature,Timestamp,TrackID,TrackName,Valence,artists
0.16300000250339508,3hYtANQYrE6pd2PbtEyTIy,0.8510000109672546,172600,0.7950000166893005,0.0,0.2070000022649765,-9.02400016784668,1,9,53,1993,0.14499999582767487,108.04900360107422,4,1664317100503,1Vtiq4XS36Y0qvFp9HRyin,El Maestro,0.8389999866485596,['Los Tigres Del Norte']
8.130000060191378E-5,2yEwvVSSSUkcLeSTNyHKh8,0.36000001430511475,322017,0.796999990940094,0.8050000071525574,0.27799999713897705,-6.623000144958496,1,2,53,1993,0.05380000174045563,171.5229949951172,3,1664317102145,4Wvnm0J3Zz9cmMdEMMghGA,Undertow,0.4740000069141388,['TOOL']
5.660000169882551E-5,6olE6TJLqED3rqDCT0FyPh,0.4819999933242798,250013,0.7229999899864197,4.3200000072829425E-4,0.09390000253915787,-10.079999923706055,0,10,53,1993,0.042500000447034836,117.0510025024414,4,1664317103151,27rDqwKojyXsjg2GmYmqDL,Frances Farmer Will Have Her Revenge On Seattle,0.5540000200271606,['Nirvana']
0.09239999949932098,7wqtxqI3eo7Gn1P7SpP6cQ,0.734000027179718,296213,0.6710000038146973,9.29000016185455E-5,0.12200000137090683,-8.538000106811523,1,5,53,1993,0.11800000071525574,173.1840057373047,4,1664317104200,5UKGE3tI0s5Lq0Puf6Fn0S,Whatta Man (feat. En Vogue) - EP Version,0.9449999928474426,"['Salt-N-Pepa', 'En Vogue']"
0.6729999780654907,5olDKSsFhhmwh8UCWwKtpq,0.503000020980835,191933,0.3440000116825104,0.38100001215934753,0.09600000083446503,-19.02400016784668,1,1,53,1993,0.03689999878406525,108.48899841308594,4,1664317105210,0m8tSPLHhjhGToD6LW0RPp,Windows,0.46799999475479126,['Chick Corea']
0.014299999922513962,3X458ddYA2YcVWuVIGGOYe,0.7799999713897705,301560,0.7099999785423279,3.49000001733657E-5,0.03660000115633011,-7.428999900817871,1,2,53,1993,0.0478999987244606,96.06700134277344,4,1664317106219,70iDnqYvxCPl3mPd3ddxgD,Another Sad Love Song,0.8579999804496765,['Toni Braxton']
0.4090000092983246,4o3Nv2BAyoZkyGaRXv4rT3,0.5799999833106995,217400,0.421999990940094,9.790000331122428E-5,0.05810000002384186,-14.461999893188477,0,5,53,1993,0.03420000150799751,134.91200256347656,4,1664317107328,3gPFiiuH80CF4rMPTB5VJ0,Fesuphanallah,0.843999981880188,['Erkin Koray']
4.93000028654933E-4,1QOmebWGB6FdFtW7Bo3F0W,0.5759999752044678,248267,0.3720000088214874,0.02370000071823597,0.2770000100135803,-12.604999542236328,1,9,53,1993,0.052299998700618744,178.00399780273438,4,1664317108339,6t7HHaRoBvEMqEs29qHpdt,Bajan,0.06030000001192093,['Gustavo Cerati']
6.830000202171504E-4,2yEwvVSSSUkcLeSTNyHKh8,0.6389999985694885,331853,0.6800000071525574,0.19900000095367432,0.11900000274181366,-6.735000133514404,1,2,53,1993,0.03350000083446503,129.0919952392578,3,1664317109345,5q4L6JyoEY3VN5pOSbgjS8,Swamp Song,0.3700000047683716,['TOOL']
0.06340000033378601,61J0BktHv7PuP3tjTPYXSX,0.5019999742507935,288893,0.6029999852180481,5.039999959990382E-4,0.2240000069141388,-9.496000289916992,0,6,53,1993,0.031099999323487282,176.19900512695312,4,1664317110352,7L93Qjqgh05ZuFflWWyTjN,Cose della vita,0.4740000069141388,['Eros Ramazzotti']


In [0]:
#Re-Arrange columns to make sure they fit the table form
songs_re_arranged = songs_consumed.select("TrackID","TrackName","Popularity","DurationMS","ArtistID","ReleaseYear","Danceability","Energy","MusicalKey","Loudness","Mode","Speechiness","Acousticness","Instrumentalness","Liveness","Valence","Tempo","TimeSignature")

In [0]:
display(songs_re_arranged)


TrackID,TrackName,Popularity,DurationMS,ArtistID,ReleaseYear,Danceability,Energy,MusicalKey,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,TimeSignature
1Vtiq4XS36Y0qvFp9HRyin,El Maestro,53,172600,3hYtANQYrE6pd2PbtEyTIy,1993,0.8510000109672546,0.7950000166893005,9,-9.02400016784668,1,0.14499999582767487,0.16300000250339508,0.0,0.2070000022649765,0.8389999866485596,108.04900360107422,4
4Wvnm0J3Zz9cmMdEMMghGA,Undertow,53,322017,2yEwvVSSSUkcLeSTNyHKh8,1993,0.36000001430511475,0.796999990940094,2,-6.623000144958496,1,0.05380000174045563,8.130000060191378E-5,0.8050000071525574,0.27799999713897705,0.4740000069141388,171.5229949951172,3
27rDqwKojyXsjg2GmYmqDL,Frances Farmer Will Have Her Revenge On Seattle,53,250013,6olE6TJLqED3rqDCT0FyPh,1993,0.4819999933242798,0.7229999899864197,10,-10.079999923706055,0,0.042500000447034836,5.660000169882551E-5,4.3200000072829425E-4,0.09390000253915787,0.5540000200271606,117.0510025024414,4
5UKGE3tI0s5Lq0Puf6Fn0S,Whatta Man (feat. En Vogue) - EP Version,53,296213,7wqtxqI3eo7Gn1P7SpP6cQ,1993,0.734000027179718,0.6710000038146973,5,-8.538000106811523,1,0.11800000071525574,0.09239999949932098,9.29000016185455E-5,0.12200000137090683,0.9449999928474426,173.1840057373047,4
0m8tSPLHhjhGToD6LW0RPp,Windows,53,191933,5olDKSsFhhmwh8UCWwKtpq,1993,0.503000020980835,0.3440000116825104,1,-19.02400016784668,1,0.03689999878406525,0.6729999780654907,0.38100001215934753,0.09600000083446503,0.46799999475479126,108.48899841308594,4
70iDnqYvxCPl3mPd3ddxgD,Another Sad Love Song,53,301560,3X458ddYA2YcVWuVIGGOYe,1993,0.7799999713897705,0.7099999785423279,2,-7.428999900817871,1,0.0478999987244606,0.014299999922513962,3.49000001733657E-5,0.03660000115633011,0.8579999804496765,96.06700134277344,4
3gPFiiuH80CF4rMPTB5VJ0,Fesuphanallah,53,217400,4o3Nv2BAyoZkyGaRXv4rT3,1993,0.5799999833106995,0.421999990940094,5,-14.461999893188477,0,0.03420000150799751,0.4090000092983246,9.790000331122428E-5,0.05810000002384186,0.843999981880188,134.91200256347656,4
6t7HHaRoBvEMqEs29qHpdt,Bajan,53,248267,1QOmebWGB6FdFtW7Bo3F0W,1993,0.5759999752044678,0.3720000088214874,9,-12.604999542236328,1,0.052299998700618744,4.93000028654933E-4,0.02370000071823597,0.2770000100135803,0.06030000001192093,178.00399780273438,4
5q4L6JyoEY3VN5pOSbgjS8,Swamp Song,53,331853,2yEwvVSSSUkcLeSTNyHKh8,1993,0.6389999985694885,0.6800000071525574,2,-6.735000133514404,1,0.03350000083446503,6.830000202171504E-4,0.19900000095367432,0.11900000274181366,0.3700000047683716,129.0919952392578,3
7L93Qjqgh05ZuFflWWyTjN,Cose della vita,53,288893,61J0BktHv7PuP3tjTPYXSX,1993,0.5019999742507935,0.6029999852180481,6,-9.496000289916992,0,0.031099999323487282,0.06340000033378601,5.039999959990382E-4,0.2240000069141388,0.4740000069141388,176.19900512695312,4


In [0]:
# Save to our spotify database

database = "landodatalakes-group4"
src_table = "dbo.Track"
user = "spotify"
password  = "T35TPA55W0RD!"

jdbcUrl = f"jdbc:sqlserver://gen10-data-fundamentals-22-07-sql-server.database.windows.net:1433;database={database};user={user};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"


songs_re_arranged.write.format("jdbc") \
  .mode("Append") \
  .option("url", jdbcUrl) \
  .option("dbtable",src_table) \
  .option("user", user) \
  .option("password", password) \
  .save()